In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity

In [7]:
path_df1 = '../data/Oct19_20/latent_df_1_with_100pct_data_50_svd_components_oct19.csv'
path_df2 = '../data/Oct19_20/latent_df_2_with_100pct_data_100_svd_components_oct19.csv'
path_df3 = '../data/Oct19_20/X_meta_with_100pct_data_oct19.csv'
df_1 = pd.read_csv(path_df1,index_col=[0])
df_2 = pd.read_csv(path_df2,index_col=[0])
meta_df = pd.read_csv(path_df3, index_col=[0])

In [8]:
df_1.head()

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
1000978,1.011997,-0.623184,-0.380176,0.486457,-0.115748,-0.212726,-0.037781,0.002280,0.096322,-0.012538,...,-0.021127,-0.148078,0.101628,0.095404,0.192290,-0.141239,-0.019931,-0.209001,0.039352,-0.287247
1001588,0.450582,-0.154148,0.113206,1.006491,0.751239,0.208610,0.073950,0.034433,0.013401,0.007141,...,-0.034642,-0.096371,-0.043830,-0.010137,-0.062971,0.124478,0.032251,0.046620,0.012112,-0.003206
1001606,1.003073,-0.623449,-0.369852,0.495251,-0.126210,-0.216949,-0.036066,0.007362,0.085205,-0.005006,...,-0.030855,-0.044015,-0.001673,-0.019508,-0.036033,0.010694,0.029710,0.062039,-0.134516,0.169932
1002042,0.466080,-0.143793,0.103378,1.014509,0.748322,0.207734,0.060810,0.026635,0.023192,-0.001750,...,-0.010798,-0.150577,0.108524,0.096416,0.194678,-0.137793,-0.018459,-0.218849,0.032258,-0.301445
1002062,0.466080,-0.143793,0.103378,1.014509,0.748322,0.207734,0.060810,0.026635,0.023192,-0.001750,...,-0.010798,-0.150577,0.108524,0.096416,0.194678,-0.137793,-0.018459,-0.218849,0.032258,-0.301445


In [9]:
df_2.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
1000978,3.457116e-12,-7.647303e-12,-5.229807e-11,-5.223009e-12,7.521721e-11,4.100482e-11,2.002182e-11,2.784693e-12,3.707997e-10,1.602469e-11,...,3.657432e-11,-2.287473e-10,3.632910e-10,3.954256e-10,-7.914660e-11,3.949634e-10,-9.391689e-11,-7.042974e-11,2.911405e-10,1.421546e-10
1001588,-3.414175e-10,-4.915963e-10,1.281209e-09,4.920853e-10,3.701399e-09,1.432284e-08,5.503048e-09,-3.178901e-09,-7.128125e-09,-8.080676e-09,...,-5.754571e-08,-1.927902e-08,4.152988e-08,1.843542e-08,2.513252e-08,4.960889e-08,-1.548731e-08,2.504717e-08,-6.208404e-09,2.985416e-08
1001606,4.432515e-12,-6.376881e-12,-3.472441e-11,3.290277e-11,-4.962909e-11,3.091941e-11,-1.784861e-11,-1.440861e-10,9.653236e-11,-2.458111e-11,...,1.754743e-10,-5.235417e-10,-8.677409e-11,6.597006e-10,6.133764e-10,2.535012e-10,-1.626082e-10,-1.763363e-10,6.570468e-10,1.994303e-10
1002042,1.235610e-11,-1.841875e-12,2.211459e-11,3.281866e-11,5.148971e-12,-1.277910e-10,-2.298846e-11,-1.277047e-10,-8.637708e-12,-9.145993e-11,...,5.049348e-10,1.240354e-10,-9.891541e-10,1.354112e-10,1.585830e-10,-2.993687e-10,-7.985069e-10,6.287094e-10,5.029197e-10,3.445676e-10
1002062,-4.436925e-12,-2.987696e-12,-5.152762e-11,-7.689036e-11,5.316302e-11,1.327261e-10,4.074853e-11,-2.240237e-11,2.214909e-10,-5.582463e-11,...,-3.713891e-10,9.041996e-13,-9.825824e-10,-6.583270e-10,1.842813e-10,6.051739e-11,3.975980e-11,-1.179401e-11,-1.539785e-10,-4.021296e-11


# Latent matrix from metadata

In [2]:
filename = '../data/10%sample/2019-Oct.csv_10%.csv'

In [3]:
!pwd

/Users/helena/code/sailormoonvicky/eCommerce/notebooks


In [4]:
import random
p = 1  # 1% of the lines
# keep the header, then take only 1% of lines
# if random from [0,1] interval is greater than 0.01 the row will be skipped
df = pd.read_csv(
         filename,
         header=0, 
         skiprows=lambda i: i>0 and random.random() > p
)
df.shape

(4248118, 9)

In [5]:
df.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-10-01 00:00:17 UTC,view,23100006,2053013561638126333,NaN,NaN,357.79,513642368,17566c27-0a8f-4506-9f30-c6a2ccbf583b
1,2019-10-01 00:00:20 UTC,view,4803399,2053013554658804075,electronics.audio.headphone,jbl,33.21,555428858,8a6afed4-77f8-40c9-8e76-e062b28216ce
2,2019-10-01 00:00:23 UTC,view,6200260,2053013552293216471,appliances.environment.air_heater,midea,47.62,538645907,7d9a8784-7b6c-426e-9924-9f688812fd71
3,2019-10-01 00:00:58 UTC,view,4802639,2053013554658804075,electronics.audio.headphone,sony,218.77,514808401,1877639d-46a4-44f8-bae9-a14456952240
4,2019-10-01 00:01:11 UTC,view,1004836,2053013555631882655,electronics.smartphone,samsung,241.19,546259103,6e2984c8-502e-4fe7-bbba-34087f760175


In [ ]:
df.category_code.nunique()

In [ ]:
df.columns

In [ ]:
df.head()

In [ ]:
def preprocessing_feat(X, drop_event_time=False):
    if drop_event_time:
        X = X.drop("event_time", axis=1)
    
    X_preprocessed = X.dropna(subset = ['category_code', 'brand']) #tbd!!
    X_preprocessed = X_preprocessed.drop_duplicates()
    X_preprocessed['category_code'] = X_preprocessed['category_code'].str.replace('.',' ')
    return X_preprocessed

In [ ]:
X_preprocessed = preprocessing_feat(df,  drop_event_time=True)

In [ ]:
X_preprocessed.shape

## Pricing criterion

In [ ]:
def make_column(row):
    if row["price"] < row["25%"]:
        return "low"
    elif row["price"] < row["75%"]:
        return "medium"
    else:
        return "high"

def pricing_criterion(X):
    pricing_guide = X.groupby('category_code')['price'].describe()[["25%", "75%"]].reset_index()
    X_merged = X.merge(pricing_guide, on="category_code", how="right")
    X_merged["price_category"] = X_merged.apply(lambda row: make_column(row), axis=1)
    return X_merged


In [ ]:
X_merged = pricing_criterion(X_preprocessed)

In [ ]:
def metadata(X):
    X['metadata'] = X[['category_code', 'brand', 'price_category']].apply(lambda x: ' '.join(x), axis = 1)
    return X


In [ ]:
X_meta = metadata(X_merged)
X_meta.nunique()

In [ ]:
X_meta.set_index(X_meta['product_id'], inplace=True)

In [ ]:
print(X_meta.shape)
X_meta.head()

### Count Vectorizer

In [ ]:
count = CountVectorizer()
count_matrix = count.fit_transform(X_meta['metadata'])
count_df_1 = pd.DataFrame(count_matrix.toarray(), index=X_meta.product_id.tolist())
print(count_df_1.shape)
count_df_1.head()

In [ ]:
count_df_1 = count_df_1.reset_index()

In [ ]:
len(list(set(count_df_1.index)))#.nunique()

In [ ]:
count_df_1.drop_duplicates(subset='index', keep='first', inplace=True) 
count_df_1["product_id"] = count_df_1["index"]
count_df_1.drop("index", axis=1, inplace=True)
count_df_1.head()

### Dimensionality reduction

In [ ]:
n = 50
svd = TruncatedSVD(n_components=n)
latent_df_1 = svd.fit_transform(count_df_1.set_index("product_id"))

In [ ]:
plt.plot(svd.explained_variance_ratio_.cumsum())
plt.xlabel('number of singular value components')
plt.ylabel('Cumulative percent of variance')   
plt.grid()
plt.show()

In [ ]:
#count_df_1=count_df_1.set_index("product_id")
#count_df_1.head()
df_for_latent1 = count_df_1.product_id.tolist()

In [ ]:
#n defined above
#n = 50
latent_df_1 = pd.DataFrame(latent_df_1[:,0:n], index=df_for_latent1)
print(latent_df_1.shape)
latent_df_1

In [ ]:
latent_df_1.reset_index().drop_duplicates().shape

In [ ]:
#X_meta['product_id'].nunique()

# Latent matrix from event types

In [ ]:
def preprocessing_event(X):
    X_preprocessed = X.dropna(subset = ['category_code', 'brand']) #tbd!!
    X_preprocessed = X_preprocessed.drop_duplicates()
    return X_preprocessed

In [ ]:
df_event = preprocessing_event(df)
df_event.user_id.nunique()

In [ ]:
dct = {'view': 1, 'cart': 3, 'purchase': 5}

In [ ]:
df_event['rating'] = df_event['event_type'].map(dct)

In [ ]:
df_event["rating"].sum()

In [ ]:
# df_rating.shape

In [ ]:
# df_rating = df_event.pivot_table(values='rating',
#                                  index='product_id',
#                                  columns='user_id',
#                                  aggfunc="sum").fillna(0)
# df_rating

In [ ]:
df_event.nunique()

In [ ]:
df_event.drop_duplicates(subset='product_id',inplace=True)
df_event.nunique()

In [ ]:
%%time

df_rating = df_event.pivot(values='rating',
               index='product_id',
               columns='user_id').fillna(0)

df_rating

In [ ]:
df_rating.sum().reset_index()[0].sum()

In [ ]:
df_event.groupby("user_id").agg({"rating":sum}).sort_values(by="rating").sum()

In [ ]:
df_event.groupby("user_id").agg({"rating":sum}).sort_values(by="rating").sum()

In [ ]:
df_rating.reset_index()

In [ ]:
svd = TruncatedSVD(n_components=75)
latent_df_2 = svd.fit_transform(df_rating)

In [ ]:
plt.plot(svd.explained_variance_ratio_.cumsum())
plt.xlabel('number of singular value components')
plt.ylabel('Cumulative percent of variance')   
plt.grid()
plt.show()

In [ ]:
latent_df_2 = pd.DataFrame(latent_df_2, index=df_rating.reset_index().product_id.tolist())
latent_df_2.head()

In [ ]:
latent_df_2.shape

# Apply Cosine Similarity in Content and Collaborative Matrices


In [ ]:
product_id=5100337

In [ ]:
# Get the latent vectors for "Toy Story" from content and collaborative matrices
v1 = np.array(latent_df_1.loc[product_id]).reshape(1, -1)
v2 = np.array(latent_df_2.loc[product_id]).reshape(1, -1)

# Compute the cosine similarity of this movie with the others in the list
sim1 = cosine_similarity(latent_df_1, v1).reshape(-1)
sim2 = cosine_similarity(latent_df_2, v2).reshape(-1)

In [ ]:
dictDf_1 = {'features': sim1} 
recommendation_df_1 = pd.DataFrame(dictDf_1, index = latent_df_1.index)
# recommendation_df_1.sort_values('features', ascending=False, inplace=True)

In [ ]:
dictDf_2 = {'ratings': sim2} 
recommendation_df_2 = pd.DataFrame(dictDf_2, index = latent_df_2.index)
# recommendation_df_2.sort_values('ratings', ascending=False, inplace=True)

In [ ]:
weight_features = 0.8

In [ ]:
recommendation_combined = pd.merge(recommendation_df_1, recommendation_df_2, left_index=True, right_index=True)
recommendation_combined['hybrid'] = ((weight_features*recommendation_combined['features'] + (1-weight_features)*recommendation_combined['ratings']))

In [ ]:
recommendation_combined.sort_values('ratings', ascending=False, inplace=True)
recommendation_combined

In [ ]:
recommendation_combined.head()

In [ ]:
def top_n_products(rec_df, meta_df, n=10, ranking='hybrid'):
    
    """Valid inouts for ranking: 'features', 'ratings', 'hybrid'"""
    
    feat_idx = rec_df.sort_values(ranking, ascending=False).index
    
    
    return feat_idx
    
#     for i in feat_idx:
#         meta_text = meta_df[meta_df['product_id'] == i][['metadata']].iloc[0,:][0]
#         lst = []
#         lst.append(meta_text)
#         return lst

In [ ]:
indexes = top_n_products(recommendation_combined, X_meta, ranking='features')

In [ ]:
indexes

In [ ]:
len(indexes)

In [ ]:
X_meta[X_meta.index.isin(indexes)]['metadata']

In [ ]:
X_meta.head()

In [ ]:
def top_n_products(rec_df, meta_df, n=10, ranking='hybrid'):
    
    """Valid inputs for ranking: 'features', 'ratings', 'hybrid'"""

    feat_idx = rec_df.sort_values(ranking, ascending=False).index
    
    counter = 0
    
    product_ids=[]
    metas=[]
    prices=[]
    
    for i in feat_idx:
        meta_text = meta_df[meta_df['product_id'] == i][['metadata']].iloc[0,:][0]
        price = meta_df[meta_df['product_id'] == i][['price']].iloc[0,:][0]
        
#         if counter == 0:
#             print(f"Top {n} recommendations for product_id {product_id}:")
#             print(f"{i} - {meta_text} {price} \n")
#             print("-----------------------")

#         else:
#             print(f"Rec {counter}) {i} - {meta_text} {price}")
        
        product_ids.append(i)
        metas.append(meta_text)
        prices.append(price)
        
        counter += 1
    
    new_df = pd.DataFrame({'product_id':product_ids, 'meta_text':metas, 'price':prices})
    
    return new_df.drop_duplicates('meta_text').iloc[:10,:]

In [ ]:
top_n_products(recommendation_combined, X_meta, ranking='features')

In [ ]:
new_df = top_n_products(recommendation_combined, X_meta, ranking='features')

In [ ]:
new_df.drop_duplicates('meta_text').iloc[:10,:]

In [ ]:
def top_n_products(rec_df, meta_df, n=10, ranking='hybrid'):
    
    """Valid inputs for ranking: 'features', 'ratings', 'hybrid'"""

    feat_idx = rec_df.sort_values(ranking, ascending=False).index
    
    counter = 0
    
    product_ids=[]
    metas=[]
    prices=[]
    
    for i in feat_idx:
        meta_text = meta_df[meta_df['product_id'] == i][['metadata']].iloc[0,:][0]
        price = meta_df[meta_df['product_id'] == i][['price']].iloc[0,:][0]
        
#         if counter == 0:
#             print(f"Top {n} recommendations for product_id {product_id}:")
#             print(f"{i} - {meta_text} {price} \n")
#             print("-----------------------")

#         else:
#             print(f"Rec {counter}) {i} - {meta_text} {price}")
        
        product_ids.append(i)
        metas.append(meta_text)
        prices.append(price)
        
        counter += 1
    
    new_df = pd.DataFrame({'product_id':product_ids, 'meta_text':metas, 'price':prices}).drop_duplicates('meta_text').iloc[:n,:]
    return new_df.reset_index().drop(columns='index')

In [ ]:
top_n_products(recommendation_combined, X_meta, ranking='features', n=10)